## 千问模型（qwen-plus）Assistant API案例

- [原文链接](https://help.aliyun.com/zh/model-studio/developer-reference/overview)

### 本文主要内容
1. Assistant API使用流程
2. 调用function call案例
3. 调用千问自带插件案例

## 0、环境准备

1、安装千问模型的SDK dashscope

In [1]:
!pip install dashscope

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


2、[阿里云百炼大模型服务平台](https://bailian.console.aliyun.com/#/home)注册账号，按照[教程](https://help.aliyun.com/zh/model-studio/getting-started/first-api-call-to-qwen)开通服务。

在`.env`文件中配置你的 `DASHSCOPE_API_KEY="YOUR_DASHSCOPE_API_KEY"`


通义千问-Plus模型有100w token的免费额度使用。

In [2]:
from http import HTTPStatus

import os
import json
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import dashscope
dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")

from dashscope import Assistants, Threads, Messages, Runs

## 1、自定义函数

In [3]:
def big_add(left, right):
    return left + right

# 可以被回调的函数放入此字典
function_mapper = {
    "big_add": big_add,
}

## 2、创建assistant

In [10]:
def create_assistant():
    assistant = Assistants.create(
    model='qwen-plus',
    name='My helper',
    description='A tool helper.',
    instructions='You are a helpful assistant. When asked a question, use tools wherever possible.',
    # tool to use, current we support text to image text_to_image, quark_search, code_interpreter, and function call.
    tools=[
        {'type': "code_interpreter"},
        {'type': "generate_qrcode"},
#         {'type': "text_to_image"},
        {
        'type': 'function',
        'function': {
                'name': 'big_add',
                'description': 'Add to number',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'left': {
                            'type': 'integer',
                            'description': 'The left operator'
                        },
                        'right': {
                            'type': 'integer',
                            'description': 'The right operator.'
                        }
                    },
                    'required': ['left', 'right']
                }
        }
    }],)
    return assistant



assistant = create_assistant()

## 3、调用assistant执行流程

In [11]:
def send_message(assistant, message='你好'):
    print(f"Query: {message}")

    # 创建Thread
    thread = Threads.create()

    # 创建Message
    message = Messages.create(thread.id, content=message)
    
    # 创建Run
    run = Runs.create(thread.id, assistant_id=assistant.id)
#     print("=====Run======")
#     print(run)
    # 等待执行完成
    run_status = Runs.wait(run.id, thread_id=thread.id)

    # 如果需要调用function calling
    if run_status.required_action:
        f = run_status.required_action.submit_tool_outputs.tool_calls[0].function
        func_name = f['name']
        param = json.loads(f['arguments'])
        print("=====function======")
        print(f)
        if func_name in function_mapper:
            output = function_mapper[func_name](**param)
        else:
            output = ""

        tool_outputs = [{
            'output':
                str(output)
        }]
        print("===== tool output =====")
        print(tool_outputs)
        
        run = Runs.submit_tool_outputs(run.id,
                                       thread_id=thread.id,
                                       tool_outputs=tool_outputs)
#         print(run)
        run_status = Runs.wait(run.id, thread_id=thread.id)
    
    # get the thread messages.
    msgs = Messages.list(thread.id)
    
    print("运行结果:")
    for message in msgs['data'][::-1]:
        print(message)
        print("content: ", message['content'][0]['text']['value'])
    print("\n")

In [13]:
send_message(assistant=assistant,message="计算 87787 和 788988737的和")

Query: 计算 87787 和 788988737的和
{
    "assistant_id": "asst_ccdb5062-32e0-4eca-a47e-989dd54d353c",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1723104278611,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_5198caa0-b94c-4cf9-a243-9b3b8a95517a",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "a7d489ee-1521-91c5-99ed-e2e33a2d85f4",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "status_code": 200,
    "temperature": null,
    "thread_id": "thread_4f0818e6-4965-4fe6-af64-32758db0fae3",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "generate_qrcode"
        },
        {
            "function": {
                "descripti

{
    "assistant_id": "asst_ccdb5062-32e0-4eca-a47e-989dd54d353c",
    "cancelled_at": null,
    "completed_at": 1723104281064,
    "created_at": 1723104278611,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_5198caa0-b94c-4cf9-a243-9b3b8a95517a",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "af0b5deb-5a0d-91f2-afe4-3880e41c9b41",
    "required_action": {
        "submit_tool_outputs": {
            "tool_calls": [
                {
                    "function": {
                        "arguments": "{\"left\": 87787, \"right\": 788988737}",
                        "name": "big_add"
                    },
                    "id": "",
                    "type": "function"
                }
            ]
        },
        "type": "submit_tool_outputs"
 

In [14]:
send_message(assistant=assistant,message="我有一组数据，想让你帮我分析。以下是一个简单的列表：[4, 8, 15, 16, 23, 42]。请计算这些数值的平均值、中位数和方差，并告诉我结果。")

Query: 我有一组数据，想让你帮我分析。以下是一个简单的列表：[4, 8, 15, 16, 23, 42]。请计算这些数值的平均值、中位数和方差，并告诉我结果。
{
    "assistant_id": "asst_ccdb5062-32e0-4eca-a47e-989dd54d353c",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1723104292720,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_98a0e82d-64a2-4992-8d31-69f6499c2843",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "5b3b2d51-3d7e-9db5-aaee-4c9ba3aea31f",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "status_code": 200,
    "temperature": null,
    "thread_id": "thread_483c8cbc-f629-4063-b57b-5fa91713eb28",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "generate_qrcode"
        },
        {

{
    "assistant_id": "asst_ccdb5062-32e0-4eca-a47e-989dd54d353c",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1723104292720,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_98a0e82d-64a2-4992-8d31-69f6499c2843",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "7bf68edf-1562-92f5-9b95-ffc44d93e806",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "status_code": 200,
    "temperature": null,
    "thread_id": "thread_483c8cbc-f629-4063-b57b-5fa91713eb28",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "generate_qrcode"
        },
        {
            "function": {
                "description": "Add to number",
        

{
    "assistant_id": "asst_ccdb5062-32e0-4eca-a47e-989dd54d353c",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1723104292720,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_98a0e82d-64a2-4992-8d31-69f6499c2843",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "cd0ed474-cf9d-9c53-9a39-228974d60b8c",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "status_code": 200,
    "temperature": null,
    "thread_id": "thread_483c8cbc-f629-4063-b57b-5fa91713eb28",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "generate_qrcode"
        },
        {
            "function": {
                "description": "Add to number",
        

In [15]:
send_message(assistant=assistant,message="将http://www.baidu.com转化为二维码")

Query: 将http://www.baidu.com转化为二维码
{
    "assistant_id": "asst_ccdb5062-32e0-4eca-a47e-989dd54d353c",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1723104367651,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_9d6f60dd-9aa6-46ac-9498-0767fcf380d8",
    "instructions": "You are a helpful assistant. When asked a question, use tools wherever possible.",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "cfb82ac5-b1b8-9f58-841e-8d721e05f2a3",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "status_code": 200,
    "temperature": null,
    "thread_id": "thread_1e370542-991c-4367-9f1c-728529a78de7",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "generate_qrcode"
        },
        {
            "function": {
                "desc

运行结果:
{'content': [{'text': {'annotations': [], 'value': '将http://www.baidu.com转化为二维码'}, 'type': 'text'}], 'id': 'message_1f400362-acbe-4690-9692-490dee39f654', 'object': 'thread.message', 'created_at': 1723104367407, 'thread_id': 'thread_1e370542-991c-4367-9f1c-728529a78de7', 'incomplete_details': {}, 'completed_at': None, 'incomplete_at': None, 'role': 'user', 'assistant_id': '', 'run_id': '', 'file_ids': [], 'metadata': {}, 'status': '', 'name': '', 'plugin_call': {}, 'tool_calls': []}
content:  将http://www.baidu.com转化为二维码
{'content': [{'text': {'annotations': [], 'value': '这是http://www.baidu.com的二维码：\n![](http://appflow-data-cn-hangzhou.oss-cn-hangzhou.aliyuncs.com/temp/qr_image/e4098fb3d2e44bcead68.png?Expires=1723111569&OSSAccessKeyId=LTAI5tCYphFhocJemeMWmXst&Signature=LThGRTVwboP%2BICMS346UPUGEcQY%3D)'}, 'type': 'text'}], 'id': 'message_884d188d-d89b-4b49-bbc9-52a5d39745ac', 'object': 'thread.message', 'created_at': 1723104370561, 'thread_id': 'thread_1e370542-991c-4367-9f1c-728